In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from numpy import random

In [2]:
events_df = pd.read_csv('./data/retailrocket/events.csv')
category_tree_df = pd.read_csv('./data/retailrocket/category_tree.csv')
item_properties_1_df = pd.read_csv('./data/retailrocket/item_properties_part1.csv')
item_properties_2_df = pd.read_csv('./data/retailrocket/item_properties_part2.csv')

In [3]:
item_properties_df = pd.concat([item_properties_1_df, item_properties_2_df])

In [4]:
item_properties_df

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
9275898,1433646000000,236931,929,n12.000
9275899,1440903600000,455746,6,150169 639134
9275900,1439694000000,347565,686,610834
9275901,1433646000000,287231,867,769062


In [5]:
events_df

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [6]:
events_df = events_df.drop(['event','transactionid'],axis=1)

不重复的用户数量

In [7]:
events_df.visitorid.nunique()

1407580

将目标放在活动交互次数大于10次的用户

In [8]:
users = events_df['visitorid']
users = users.value_counts()
users = users.iloc[users.values > 10].index
users = users.sort_values()
users

Int64Index([     54,      75,     155,     172,     224,     270,     351,
                420,     549,     604,
            ...
            1406881, 1406914, 1406941, 1407091, 1407169, 1407206, 1407281,
            1407340, 1407374, 1407512],
           dtype='int64', length=19582)

In [9]:
dataset = pd.read_csv('./data/retailrocket/SLdata.csv')
dataset

,timestamp,visitorid,itemid,nextitemid
0,1442338531705,54,388096,283115
1,1442338665550,54,283115,38965
2,1442338748248,54,38965,319680
3,1442338841333,54,319680,283115
4,1442339111712,54,283115,388096
...,...,...,...,...
527127,1432954132591,1407512,118079,398091
527128,1432954161462,1407512,398091,398091
527129,1432954196910,1407512,398091,241408
527130,1432954247297,1407512,241408,398533


In [10]:
group = dataset.groupby('visitorid',group_keys=False).groups
group

{54: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 75: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], 155: [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], 172: [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84], 224: [85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95], 270: [96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107], 351: [108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123], 420: [124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136], 549: [137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152], 604: [153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166], 627: [167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184], 648: [185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], 724: [200, 201, 202, 203, 2

In [11]:
item = dataset['itemid'][0]
item

388096

做出每个用户访问物品的列表

In [12]:
user_list=[]
action_list=[]
for user in dataset.visitorid.unique().tolist():
    item_list=[]
    for item in group[user]:
        item = dataset['itemid'][item]
        item_list.append(item)
    user_list.append(user)
    action_list.append(item_list)

In [13]:
user_list

[54,
 75,
 155,
 172,
 224,
 270,
 351,
 420,
 549,
 604,
 627,
 648,
 724,
 772,
 795,
 877,
 878,
 1014,
 1115,
 1136,
 1262,
 1344,
 1346,
 1581,
 1634,
 1722,
 1765,
 1879,
 2004,
 2019,
 2110,
 2114,
 2125,
 2194,
 2242,
 2304,
 2326,
 2371,
 2432,
 2512,
 2519,
 2527,
 2656,
 2783,
 2798,
 2799,
 2861,
 2966,
 3013,
 3064,
 3090,
 3104,
 3106,
 3242,
 3258,
 3396,
 3403,
 3440,
 3471,
 3493,
 3741,
 3868,
 3879,
 3905,
 3926,
 3936,
 3971,
 4024,
 4101,
 4113,
 4256,
 4408,
 4413,
 4536,
 4547,
 4554,
 4739,
 4886,
 4909,
 4959,
 5042,
 5117,
 5207,
 5235,
 5272,
 5344,
 5377,
 5630,
 5949,
 5967,
 6029,
 6167,
 6170,
 6399,
 6410,
 6468,
 6565,
 6584,
 6699,
 6704,
 6770,
 6952,
 6958,
 7030,
 7127,
 7166,
 7189,
 7542,
 7568,
 7682,
 7845,
 7950,
 7962,
 7969,
 8006,
 8276,
 8293,
 8299,
 8316,
 8359,
 8360,
 8393,
 8411,
 8495,
 8504,
 8540,
 8644,
 8670,
 8711,
 8756,
 8903,
 8914,
 8916,
 8958,
 8972,
 9004,
 9087,
 9145,
 9188,
 9385,
 9442,
 9535,
 9863,
 10023,
 10122,
 1

In [14]:
action_list

[[388096,
  283115,
  38965,
  319680,
  283115,
  388096,
  319680,
  442228,
  442228,
  249114],
 [257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  96883,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575,
  257575],
 [134620,
  123027,
  134620,
  50928,
  373637,
  151670,
  143373,
  368372,
  452082,
  181405,
  41882,
  442601,
  224623],
 [397068,
  55710,
  464731,
  403998,
  18519,
  27248,
  363106,
  10034,
  302810,
  10034,
  412628,
  374400,
  323030,
  464731,
  10034,
  378862,
  292240,
  254301,
  464731,
  253720,
  10034,
  292240,
  292240,
  292240,
  228229,
  10034,
  10034,
  62300,
  202268,
  107943,
  79918,
  465522,
  465522,
  465522,
  465522,
  465522,
  465522],
 [443407,
  168697,
  443407,
  168697,
  443407,
  168697,
  443407,
  168697,
  336432,
  168697,
  443407],
 [286735,
  286735,
  286735,
  286735,
  286735,


In [15]:
data = {'user':user_list, 'actions':action_list}
df = pd.DataFrame(data)
df

,user,actions
0,54,"[388096, 283115, 38965, 319680, 283115, 388096..."
1,75,"[257575, 257575, 257575, 257575, 257575, 25757..."
2,155,"[134620, 123027, 134620, 50928, 373637, 151670..."
3,172,"[397068, 55710, 464731, 403998, 18519, 27248, ..."
4,224,"[443407, 168697, 443407, 168697, 443407, 16869..."
...,...,...
19577,1407206,"[384514, 162046, 162046, 166941, 454393, 19838..."
19578,1407281,"[69868, 116624, 264521, 174409, 288340, 260581..."
19579,1407340,"[321676, 66145, 148153, 85858, 274436, 248140,..."
19580,1407374,"[399031, 349846, 454301, 397429, 422992, 13997..."


In [16]:
df.to_csv('./data/retailrocket/seqData.csv',index=False)

In [24]:
def seq_data_iter_random(corpus, batch_size, num_steps):  #@save
    """使用随机抽样生成一个小批量子序列"""
    # 减去1，是因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    # 长度为num_steps的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)

    def data(pos):
        # 返回从pos位置开始的长度为num_steps的序列
        return corpus[pos: pos + num_steps]

    num_batches = num_subseqs // batch_size
    for i in range(0, batch_size * num_batches, batch_size):
        # 在这里，initial_indices包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield tf.constant(X), tf.constant(Y)
        #yield X, Y

In [25]:
df.index

RangeIndex(start=0, stop=19582, step=1)

In [23]:
def dataset_loader(df, batch_size, num_steps):
    for index in df.index:
        seq_data = df.loc[index].actions
        x_list=[]
        y_list=[]
        for X,Y in seq_data_iter_random(seq_data, batch_size, num_steps):
            x_list.append(X)
            y_list.append(Y)
        yield index, [x_list, y_list]        

In [28]:
def dataset_loader(df, batch_size, num_steps):
    for index in df.index:
        seq_data = df.loc[index].actions
        action_and_label = seq_data_iter_random(seq_data, batch_size, num_steps)
        yield index, action_and_label      

In [ ]:
df.head()

In [31]:
for user, actions in dataset_loader(df.head(), 1, 9):
    print('userid:',user)
    for X,Y in actions:
        print('X:',X, '\nY:',Y)

userid: 0
X: tf.Tensor([[388096 283115  38965 319680 283115 388096 319680 442228 442228]], shape=(1, 9), dtype=int32) 
Y: tf.Tensor([[283115  38965 319680 283115 388096 319680 442228 442228 249114]], shape=(1, 9), dtype=int32)
userid: 1
X: tf.Tensor([[257575 257575 257575 257575 257575 257575 257575 257575 257575]], shape=(1, 9), dtype=int32) 
Y: tf.Tensor([[257575 257575 257575 257575 257575 257575 257575 257575 257575]], shape=(1, 9), dtype=int32)
X: tf.Tensor([[257575 257575  96883 257575 257575 257575 257575 257575 257575]], shape=(1, 9), dtype=int32) 
Y: tf.Tensor([[257575  96883 257575 257575 257575 257575 257575 257575 257575]], shape=(1, 9), dtype=int32)
userid: 2
X: tf.Tensor([[134620 123027 134620  50928 373637 151670 143373 368372 452082]], shape=(1, 9), dtype=int32) 
Y: tf.Tensor([[123027 134620  50928 373637 151670 143373 368372 452082 181405]], shape=(1, 9), dtype=int32)
userid: 3
X: tf.Tensor([[ 10034 412628 374400 323030 464731  10034 378862 292240 254301]], shape=(1, 9

In [37]:
item_size = 417053

In [32]:
class RNNModel(tf.keras.layers.Layer):
    def __init__(self, rnn_layer, item_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.dense = tf.keras.layers.Dense(32)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.item_embed = tf.keras.layers.Embedding(
            input_dim=item_size, output_dim=32
        )
        # self.state_embed = tf.keras.layers.Embedding(
        #     input_dim=1, output_dim=32
        # )


    def call(self, inputs, state):
        X = inputs
        #X = self.item_embed(inputs)
        # rnn返回两个以上的值
        Y, *state = self.rnn(X, state)
        output_ = self.dropout(Y)
        output = self.dense(output_)
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.cell.get_initial_state(*args, **kwargs)

In [35]:
from CFN_impl import CFNCell

In [50]:
embed = tf.keras.layers.Embedding(
    input_dim=item_size, 
    output_dim=32,
    embeddings_initializer=tf.keras.initializers.GlorotNormal(),
    input_length=1
)

In [51]:
item_embedded = embed(item_properties_df.itemid.unique())

In [46]:
lookup = tf.nn.embedding_lookup

In [47]:
item_embedded.shape

TensorShape([417053, 32])

In [52]:
item_embedded[1]

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 3.1332532e-03,  4.3652477e-03, -1.3055919e-03,  2.7467282e-03,
       -8.2035275e-04, -2.0059447e-03, -4.4736774e-03, -5.3099170e-04,
        6.3199183e-04, -4.1392772e-04,  3.0698064e-03, -1.1123115e-03,
        1.6145589e-03,  1.1091051e-03, -6.1285816e-04,  2.6468437e-03,
       -2.3650457e-03, -2.1925967e-03, -1.3849181e-03, -2.8755611e-03,
       -1.9928198e-03, -2.9811601e-03,  2.5711413e-03, -1.0708885e-03,
       -2.0056285e-03,  1.0171937e-05, -1.2304245e-03,  1.1387934e-03,
       -4.3228436e-03, -8.6077000e-04,  2.8340839e-04,  1.8405616e-03],
      dtype=float32)>

In [ ]:
for X, Y in dataset_loader(df.head(), 1, 9):
    print(lookup(item_embedded, X))

# 训练过程

In [33]:
from d2l import tensorflow as d2l
import math

In [38]:
cfn_cell = CFNCell(256)
cfn_layer = tf.keras.layers.RNN(cfn_cell,
                                return_state=True,
                                return_sequences=True)
net = RNNModel(cfn_layer, item_size)


In [39]:
def grad_clipping(grads, theta):  #@save
    """裁剪梯度"""
    theta = tf.constant(theta, dtype=tf.float32)
    new_grad = []
    for grad in grads:
        if isinstance(grad, tf.IndexedSlices):
            new_grad.append(tf.convert_to_tensor(grad))
        else:
            new_grad.append(grad)
    norm = tf.math.sqrt(sum((tf.reduce_sum(grad ** 2)).numpy()
                        for grad in new_grad))
    norm = tf.cast(norm, tf.float32)
    if tf.greater(norm, theta):
        for i, grad in enumerate(new_grad):
            new_grad[i] = grad * theta / norm
    else:
        new_grad = new_grad
    return new_grad


In [40]:
#@save
def train_epoch_ch8(net, train_iter, loss, updater, item_embedded, lookup, use_random_iter):
    """训练模型一个迭代周期(定义见第8章)"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        X = lookup(item_embedded, X)
        Y = lookup(item_embedded, Y)
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], dtype=tf.float32)
        with tf.GradientTape(persistent=True) as g:
            y_hat, state = net(X, state)
            y = Y
            l = loss(y, y_hat)
        params = net.trainable_variables
        grads = g.gradient(l, params)
        #grads = grad_clipping(grads, 1)
        updater.apply_gradients(zip(grads, params))
        # Keras默认返回一个批量中的平均损失
        metric.add(l , 1)
        print('immediate loss:',l)
    return metric[0] / metric[1], metric[1] / timer.stop()


In [41]:
#@save
def train_ch8(net, train_iter, lr, num_epochs, strategy, item_embedded, lookup,
              use_random_iter=False):
    """训练模型(定义见第8章)"""
    with strategy.scope():
        loss = tf.keras.losses.MeanSquaredLogarithmicError()
        updater = tf.keras.optimizers.SGD(lr)
    #animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
    #                        legend=['train'], xlim=[10, num_epochs])
    #predict = lambda prefix: predict_ch8(prefix, 50, net, vocab)
    # 训练和预测
    for epoch in range(num_epochs):
        avg_loss, speed = train_epoch_ch8(net, train_iter, loss, updater, item_embedded, lookup,
                                     use_random_iter)
        # if (epoch + 1) % 10 == 0:
        #     #print(predict('time traveller'))
        #     animator.add(epoch + 1, [ppl])
        device = d2l.try_gpu()._device_name
        print(f'avg_loss {avg_loss:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    #print(predict('time traveller'))
    #print(predict('traveller'))


In [42]:
train_iter = dataset_loader(df, 1, 9)

In [43]:
device_name = d2l.try_gpu()._device_name
strategy = tf.distribute.OneDeviceStrategy(device_name)

In [ ]:
train_ch8(net, train_iter, 1, 10, strategy, item_embedded, lookup, True)